In [1]:
# %load nbheader.py
%reload_ext autoreload
%autoreload 2

from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.functions import col as S
from pyspark.sql import DataFrame, Row, Window
import os
import sys
import json
import datetime
import re
import pandas as pd
import numpy as np

In [2]:
# Intialise Spark session for GraphFrame, use equivalent of pyspark --packages io.graphframes:graphframes-spark4_2.13:0.9.2
spark = SparkSession.builder \
    .master("local[4]") \
    .config("spark.jars.packages", "io.graphframes:graphframes-spark4_2.13:0.9.2") \
    .getOrCreate()
# spark = SparkSession.builder.master("local[4]").getOrCreate()
spark.getActiveSession()
# spark.stop()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/08 17:02:15 WARN Utils: Your hostname, RCBM8368-DIII.local, resolves to a loopback address: 127.0.0.1; using 10.250.32.131 instead (on interface en0)
25/10/08 17:02:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/pmolnar/.base/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/pmolnar/.ivy2.5.2/cache
The jars for the packages stored in: /Users/pmolnar/.ivy2.5.2/jars
io.graphframes#graphframes-spark4_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-99d6603e-8e63-42f6-9bcd-d74f83443ef8;1.0
	confs: [default]
	found io.graphframes#graphframes-spark4_2.13;0.9.2 in central
:: resolution report :: resolve 58ms :: artifacts dl 2ms
	:: modules in use:
	io.graphframes#graphframes-spark4_2.13;0.9.2 from central in [d

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63540)
Traceback (most recent call last):
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/pmolnar/homebrew/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/pmolnar/.b

In [10]:
! mkdir -p ./tmp/graphframes-checkpoints

In [3]:
from graphframes import GraphFrame

spark.sparkContext.setCheckpointDir("./tmp/graphframes-checkpoints")

In [7]:
# Install graphframes-py and GraphFrames jar if running in Colab or new environment
# !pip install graphframes-py

# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, lit


# 1. Prepare vertices and edges DataFrames
vertices = spark.createDataFrame([
    ("a", "Alice"),
    ("b", "Bob"),
    ("c", "Charlie"),
    ("d", "David")
], ["id", "name"])

edges = spark.createDataFrame([
    ("a", "b", "knows"),
    ("b", "c", "knows"),
    ("c", "a", "knows"),
    ("a", "d", "knows")
], ["src", "dst", "relationship"])

# 2. Create GraphFrame
g = GraphFrame(vertices, edges)

# 3. Run PageRank example
results = g.pageRank(resetProbability=0.15, maxIter=5)
results.vertices.select("id", "name", "pagerank").orderBy(S("pagerank").desc()).show()

+---+-------+------------------+
| id|   name|          pagerank|
+---+-------+------------------+
|  a|  Alice| 1.313776242780802|
|  c|Charlie|1.0007283492186585|
|  d|  David|0.8427477040002699|
|  b|    Bob|0.8427477040002699|
+---+-------+------------------+



In [8]:
# 4. Find Connected Components
## fix this: components = g.connectedComponents()
components = g.connectedComponents()
components.select("id", "component").show()

+---+------------+
| id|   component|
+---+------------+
|  a|807453851648|
|  b|807453851648|
|  c|807453851648|
|  d|807453851648|
+---+------------+



25/10/08 17:03:02 WARN ConnectedComponents$: The DataFrame returned by ConnectedComponents is persisted and loaded.


In [9]:
# 5. Find triangle counts (clustering)
triangles = g.triangleCount()
triangles.select("id", "count").show()

# 6. Motif finding: look for mutual relationships
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
motifs.show()

+---+-----+
| id|count|
+---+-----+
|  a|    1|
|  b|    1|
|  c|    1|
|  d|    0|
+---+-----+

+---+---+---+---+
|  a|  e|  b| e2|
+---+---+---+---+
+---+---+---+---+



In [10]:

# Create vertices DataFrame: user id, name, and age
vertices = spark.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 28),
    ("c", "Charlie", 31),
    ("d", "David", 22)
], ["id", "name", "age"])

# Create edges DataFrame: who liked whose post
edges = spark.createDataFrame([
    ("a", "b", "likes"),
    ("b", "a", "likes"),
    ("a", "c", "likes"),
    ("c", "a", "likes"),
    ("b", "c", "likes"),
    ("c", "b", "likes"),
    ("d", "a", "likes")  # David likes Alice only (non-mutual)
], ["src", "dst", "relationship"])

# Build the graph
g = GraphFrame(vertices, edges)

# Motif query: look for mutual 'likes' relationships
motifs = g.find("(u1)-[e1]->(u2); (u2)-[e2]->(u1)")

# Filter for cases where either user is older than 30
filtered = motifs.filter("u1.age > 30 or u2.age > 30")

filtered.show()



+----------------+-------------+----------------+-------------+
|              u1|           e1|              u2|           e2|
+----------------+-------------+----------------+-------------+
|    {b, Bob, 28}|{b, a, likes}|  {a, Alice, 34}|{a, b, likes}|
|{c, Charlie, 31}|{c, a, likes}|  {a, Alice, 34}|{a, c, likes}|
|  {a, Alice, 34}|{a, b, likes}|    {b, Bob, 28}|{b, a, likes}|
|{c, Charlie, 31}|{c, b, likes}|    {b, Bob, 28}|{b, c, likes}|
|  {a, Alice, 34}|{a, c, likes}|{c, Charlie, 31}|{c, a, likes}|
|    {b, Bob, 28}|{b, c, likes}|{c, Charlie, 31}|{c, b, likes}|
+----------------+-------------+----------------+-------------+



In [11]:
filtered.printSchema()

root
 |-- u1: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: long (nullable = true)
 |-- e1: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- relationship: string (nullable = true)
 |-- u2: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: long (nullable = true)
 |-- e2: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- relationship: string (nullable = true)



In [12]:
filtered.select("u1.name", "u1.age", "u2.name", "u2.age").show()


+-------+---+-------+---+
|   name|age|   name|age|
+-------+---+-------+---+
|    Bob| 28|  Alice| 34|
|Charlie| 31|  Alice| 34|
|  Alice| 34|    Bob| 28|
|Charlie| 31|    Bob| 28|
|  Alice| 34|Charlie| 31|
|    Bob| 28|Charlie| 31|
+-------+---+-------+---+



In [13]:
# 7. Run BFS example
paths = g.bfs("name = 'Alice'", "name = 'Charlie'")

In [15]:
paths.printSchema()

root
 |-- from: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: long (nullable = true)
 |-- e0: struct (nullable = false)
 |    |-- src: string (nullable = true)
 |    |-- dst: string (nullable = true)
 |    |-- relationship: string (nullable = true)
 |-- to: struct (nullable = false)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- age: long (nullable = true)



In [18]:
paths.toPandas()

,from,e0,to
0,"(a, Alice, 34)","(a, c, likes)","(c, Charlie, 31)"


In [22]:
spark.stop()